In [3]:
#This Code predict Label for SVF Issues with Title

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailsSVF_20102019.csv')
data = data.sample(frac=1)

y = list(data['Label']) 
x = list(data['Title']) 

le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)


x_enc = x
y_enc = encode(le, y)

#80% / 20% train / test split:

train_size = int(len(x) * .8)

x_train = np.asarray(x_enc[:train_size])
y_train = np.asarray(y_enc[:train_size])

x_test = np.asarray(x_enc[train_size:])
y_test = np.asarray(y_enc[train_size:])

from keras.layers import Input, Lambda, Dense
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
adam = Adam(lr=0.04)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=10, batch_size=32)
    model.save_weights('./SVF_T_Label_Predict.h5')

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./SVF_T_Label_Predict.h5')  
    predicts = model.predict(x_test, batch_size=32)  

y_test = np.asarray(y_enc[train_size:])
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Epoch 1/10
73/73 [==============================] - 5s 73ms/step - loss: 1.6287 - acc: 0.5068
Epoch 2/10
73/73 [==============================] - 1s 8ms/step - loss: 1.1586 - acc: 0.6712
Epoch 3/10
73/73 [==============================] - 1s 8ms/step - loss: 0.9166 - acc: 0.6164
Epoch 4/10
73/73 [==============================] - 1s 8ms/step - loss: 0.6450 - acc: 0.7260
Epoch 5/10
73/73 [==============================] - 1s 8ms/step - loss: 0.4254 - acc: 0.8219
Epoch 6/10
73/73 [==============================] - 1s 8ms/step - loss: 0.3738 - acc: 0.8493
Epoch 7/10
73/73 [==============================] - 1s 8ms/step - loss: 0.2230 - acc: 0.9315
Epoch 8/10
73/73 [==============================] - 1s 8ms/step - loss: 0.1753 - acc: 0.9589
Epoch 9/10
73/73 [==============================] - 1s 8ms/step - loss: 0.0786 - acc: 0.9726
Epoch 10/10
73/73 [==============================] - 1s 8ms/step - loss: 0.0450 - acc: 1.0000
              precision    recall  f1-score   support

         bug 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# New Section